In [9]:
import gensim
word2vec_path = "/ilab/users/kc1026/Documents/cs543/GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [10]:
score_tr = []
score_ts = []

In [20]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
import gensim
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import codecs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

stop_words = set(stopwords.words('english'))
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 175303 
MAX_SEQUENCE_LENGTH = 150 # max number of words in a comment to use



In [21]:
df = pd.read_csv("/ilab/users/kc1026/Documents/cs543/emoji.csv", sep=',', header=0)
train, test = train_test_split(df, test_size=0.1)
df.count()

text     7480
label    7480
dtype: int64

In [22]:
tokenizer = RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x: re.sub("[^a-zA-Z']", ' ', x))
test['text'] = test['text'].apply(lambda x: re.sub("[^a-zA-Z']", ' ', x))

/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# all_training_words = [word for token in train["token"] for word in token]
# training_sentence_lengths = [len(token) for token in train["token"]]
# TRAINING_VOCAB = sorted(list(set(all_training_words)))
# print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
# print("Max sentence length is %s" % max(training_sentence_lengths))

In [24]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(train["text"].tolist())
x_training_sequences = tokenizer.texts_to_sequences(train["text"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

# set max length of sequences, now all data has the same length of 300
train_cnn_data = pad_sequences(x_training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

num_words = len(train_word_index) + 1
train_embedding_weights = np.zeros((num_words, EMBEDDING_DIM))
for word,index in train_word_index.items():
    if word in word2vec:
        train_embedding_weights[index,:] = word2vec[word]     


Found 8522 unique tokens.


In [25]:
test_sequences = tokenizer.texts_to_sequences(test["text"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = test_cnn_data
y_test = np_utils.to_categorical(test['label'].values)

In [26]:
x_train = train_cnn_data
y_train = np_utils.to_categorical(train['label'].values)

In [27]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

#training params
batch_size = 128 
num_epochs = 6 


model = Sequential()
embedding_layer = Embedding(num_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          2556900   
_________________________________________________________________
flatten_2 (Flatten)          (None, 45000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               11520256  
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 1799      
Total params: 14,078,955
Trainable params: 14,078,955
Non-trainable params: 0
_________________________________________________________________


In [28]:
import time
start = time.time()
model.fit(x_train, y_train, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)
end = time.time()

Train on 6058 samples, validate on 674 samples
Epoch 1/6
6058/6058 [==============================] - 9s 1ms/step - loss: 2.6474 - acc: 0.1741 - val_loss: 1.9107 - val_acc: 0.2433
Epoch 2/6
6058/6058 [==============================] - 6s 1ms/step - loss: 1.5753 - acc: 0.4473 - val_loss: 1.5250 - val_acc: 0.4392
Epoch 3/6
6058/6058 [==============================] - 6s 971us/step - loss: 0.9024 - acc: 0.7309 - val_loss: 1.4107 - val_acc: 0.4970
Epoch 4/6
6058/6058 [==============================] - 6s 958us/step - loss: 0.4229 - acc: 0.8919 - val_loss: 1.3534 - val_acc: 0.5401
Epoch 5/6
6058/6058 [==============================] - 6s 1ms/step - loss: 0.1865 - acc: 0.9581 - val_loss: 1.5176 - val_acc: 0.5178
Epoch 6/6
6058/6058 [==============================] - 7s 1ms/step - loss: 0.0815 - acc: 0.9835 - val_loss: 3.6080 - val_acc: 0.3234


In [29]:
score_tr.append(model.evaluate(x_train, y_train, verbose=0))
score_ts.append(model.evaluate(x_test, y_test, verbose=0))

In [30]:
score_tr

[[0.9412121892471222, 0.7011289364407619]]

In [31]:
score_ts

[[3.874503352425315, 0.3155080213903743]]

In [32]:
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt



plt.plot([2, 4, 6, 8, 10], np.array(score_tr)[:,0], label="Train")
plt.plot([2, 4, 6, 8, 10], np.array(score_ts)[:,0], label="Test")

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.title("Loss with Diff Epoch")
plt.legend()

plt.show()

In [33]:
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt



plt.plot([2, 4, 6, 8, 10], np.array(score_tr)[:,1], label="Train")
plt.plot([2, 4, 6, 8, 10], np.array(score_ts)[:,1], label="Test")

plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title("Accuracy with Diff Epoch")
plt.legend()

plt.show()

In [34]:
# y_predict = model.predict(x_test, batch_size=64, verbose=1)
# y_actual = test['label'].tolist()
# from sklearn.metrics import accuracy_score
# y_predict_top1 = []
# for i in range(0, len(y_predict)):
#     max_index = 0
#     max_value = 0.0
#     for j in range(0, len(y_predict[0])):
#         if y_predict[i][j] > max_value:
#             max_value = y_predict[i][j]
#             max_index = j

#     y_predict_top1.append(max_index)

# accuracy = accuracy_score(y_actual, y_predict_top1)


In [ ]:
# y_predict = model.predict(x_test, batch_size=64, verbose=1)
# y_actual = test['label'].tolist()
# from sklearn.metrics import accuracy_score
# y_predict_top1 = []
# for i in range(0, len(y_predict)):
#     max_index = 0
#     max_value = 0.0
#     for j in range(0, len(y_predict[0])):
#         if y_predict[i][j] > max_value:
#             max_value = y_predict[i][j]
#             max_index = j

#     y_predict_top1.append(max_index)

# accuracy = accuracy_score(y_actual, y_predict_top1)


In [ ]:
# accuracy